In [79]:
from glob import glob
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
root = "REPORT HL/*"
report = pd.DataFrame({})
report["CTT"] = []


In [ ]:
count = 0

list_throughput = []
list_response_time = []
list_data = []

report = pd.DataFrame(columns=["CTT", "Throughput", "Response Time"])

for val in sorted(glob(root)):
    data = pd.read_csv(val)

    duration = (data["timeStamp"].max() - data["timeStamp"].min())/1000 
    throughput = data[data["responseMessage"] == "OK"].shape[0]/(duration)
    response_time = data["elapsed"].mean()/1000

    list_throughput.append(throughput)
    list_response_time.append(response_time)

    count += 1

    if count == 3: 
        mean_throughput = sum(list_throughput) / len(list_throughput)
        mean_response_time = sum(list_response_time) / len(list_response_time)

        number = re.findall(r'\d+', val)

        new_row = pd.DataFrame({
            "CTT": [number[0]],
            "Throughput": [mean_throughput],
            "Response Time": [mean_response_time]
        })

        report = pd.concat([report, new_row], ignore_index=True)

        list_throughput = []
        list_response_time = []
        count = 0

report["CTT"] = report["CTT"].astype(int)
report = report.sort_values("CTT")
report["Power"] = report["Throughput"] / report["Response Time"]

max_throughput_idx = report["Throughput"].idxmax()
usable_capacity_ctt = report.loc[max_throughput_idx, "CTT"]

max_power_idx = report["Power"].idxmax()
knee_capacity_ctt = report.loc[max_power_idx, "CTT"]

usable_capacity_throughput = report.loc[report['CTT'] == usable_capacity_ctt, 'Throughput'].values[0]
knee_capacity_throughput = report.loc[report['CTT'] == knee_capacity_ctt, 'Throughput'].values[0]
usable_capacity_response_time = report.loc[report['CTT'] == usable_capacity_ctt, 'Response Time'].values[0]
knee_capacity_response_time = report.loc[report['CTT'] == knee_capacity_ctt, 'Response Time'].values[0]
usable_capacity_power = report.loc[report['CTT'] == usable_capacity_ctt, 'Power'].values[0]
knee_capacity_power = report.loc[report['CTT'] == knee_capacity_ctt, 'Power'].values[0]
# threshold = None
# for i in range(1, len(report)):
#    if report.iloc[i]["Response Time"] > 2 * report.iloc[i - 1]["Response Time"]:
#        threshold = report.iloc[i]["CTT"]
#        break

#response_time_limit = 220  

#report_m = report[report["Throughput"] < 200]

#max_throughput = report_m[report_m["Response Time"] <= response_time_limit]["Throughput"].max()
#max_throughput_ctt = report_m[report_m["Throughput"] == max_throughput]["CTT"].values[0]

print(f"The Usable capacity as CTT value is: {usable_capacity_ctt}")
print(f"The Knee Capacity as CTT value is: {knee_capacity_ctt}")
report

In [ ]:
plt.figure(figsize=(10, 5))
#sns.lineplot(x="CTT", y="Throughput", data=report, marker="o")
#plt.xticks([_ for _ in range(0,25000,2500)])
#plt.xticks(fontsize=8) 
plt.plot(report['CTT'], report['Throughput'], marker="o") 
plt.axvline(x = usable_capacity_ctt, color='r', linestyle='--', label="Usable Capacity")
plt.axvline(x = knee_capacity_ctt, color='g', linestyle='--', label="Knee Capacity")
plt.title("Throughput per CTT")
plt.xlabel("CTT")
plt.ylabel("Throughput (richieste/secondo)")
plt.xlim(report['CTT'].min() - 1000, report['CTT'].max() + 1000)
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 5))
#sns.lineplot(x="CTT", y="Response Time", data=report, marker="o")
#plt.xticks([_ for _ in range(0,25000,2500)])
plt.plot(report['CTT'], report['Response Time'], marker="o") 
plt.axvline(x = usable_capacity_ctt, color='r', linestyle='--', label="Usable Capacity")
plt.axvline(x = knee_capacity_ctt, color='g', linestyle='--', label="Knee Capacity")
plt.title("Response Time per CTT")
plt.xlabel("CTT")
plt.ylabel("Response Time (ms)")
plt.xlim(report['CTT'].min() - 1000, report['CTT'].max() + 1000)
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 5))
#sns.lineplot(x="CTT", y="Power", data=report, marker="o")
#plt.xticks([_ for _ in range(0,25000,2500)])
plt.plot(report['CTT'], report['Power'], marker="o") 

plt.title("Power per CTT")
plt.xlabel("CTT")
plt.ylabel("Power")
plt.xlim(report['CTT'].min() - 1000, report['CTT'].max() + 1000)
plt.grid(True)
plt.show()

In [ ]:
root = "REPORT LL/*"

list_mean = []
count = 0
report_ll = pd.DataFrame({})

for val in sorted(glob(root)):
    data = pd.read_csv(val, delim_whitespace=True, skiprows=1)
    mean_values = data.mean()

    list_mean.append(mean_values)

    count += 1

    if count == 3:
        ser = pd.Series(list_mean[0])
        
        for rec in list_mean[1:]:
            ser = ser.add(rec)

        ser = ser / 3

        number = re.findall(r'\d+', val)

        df_ll = pd.DataFrame([ser])
        df_ll["CTT"] = number[0]

        report_ll = pd.concat([report_ll, df_ll], ignore_index=True)

        list_mean = []
        count = 0

report_ll["CTT"] = report_ll["CTT"].astype(int)
report_ll = report_ll.sort_values("CTT")
report_ll

In [ ]:
plt.figure(figsize=(10, 5))
# sns.lineplot(data=report_ll[['swpd', 'free', 'buff', 'cache']])
# plt.xticks(range(len(report_ll)), report_ll['CTT'])
plt.plot(report_ll['CTT'], report_ll['swpd'], label='Virtual Memory')
plt.plot(report_ll['CTT'], report_ll['free'], label='Free Memory')
plt.plot(report_ll['CTT'], report_ll['buff'], label='Buffer Memory')
plt.plot(report_ll['CTT'], report_ll['cache'], label='Cache Memory')

plt.xlim(report_ll['CTT'].min(), report_ll['CTT'].max())
plt.xlabel("CTT")
plt.ylabel("Kb")
# plt.legend(['swpd', 'free', 'buff', 'cache'])
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
#sns.lineplot(data=report_ll[['si', 'so']])
#plt.xticks(range(len(report_ll)), report_ll['CTT'])
plt.plot(report_ll['CTT'], report_ll['si'], label='Swapped-in Memory')
plt.plot(report_ll['CTT'], report_ll['so'], label='Swapped-out Memory')

plt.xlim(report_ll['CTT'].min(), report_ll['CTT'].max())

plt.xlabel("CTT")
plt.ylabel("Kb/s")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
#sns.lineplot(data=report_ll[['bi', 'bo']])
#plt.xticks(range(len(report_ll)), report_ll['CTT'])
plt.plot(report_ll['CTT'], report_ll['bi'], label='Blocks received')
plt.plot(report_ll['CTT'], report_ll['bo'], label='Block sent')

plt.xlim(report_ll['CTT'].min(), report_ll['CTT'].max())

plt.xlabel("CTT")
plt.ylabel("blocks/s")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
#sns.lineplot(data=report_ll[['in', 'cs']])
#plt.xticks(range(len(report_ll)), report_ll['CTT'])
plt.plot(report_ll['CTT'], report_ll['in'], label='Interrupt per second')
plt.plot(report_ll['CTT'], report_ll['cs'], label='Context switches per second')

plt.xlim(report_ll['CTT'].min(), report_ll['CTT'].max())

plt.xlabel("CTT")
#plt.legend(['Interrupt per second', 'Context switches per second'])
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

plt.figure(figsize=(10, 5))
#sns.lineplot(data=report_ll[['us', 'sy', 'id', 'wa']])
#plt.xticks(range(len(report_ll)), report_ll['CTT'])
plt.plot(report_ll['CTT'], report_ll['us'], label='User')
plt.plot(report_ll['CTT'], report_ll['sy'], label='Kernel')
plt.plot(report_ll['CTT'], report_ll['id'], label='Idle')
plt.plot(report_ll['CTT'], report_ll['wa'], label='Wait')

plt.xlim(report_ll['CTT'].min(), report_ll['CTT'].max())

plt.xlabel("CTT")
plt.ylabel("%")

plt.legend()
plt.grid(True)
plt.show()


In [84]:
root = "REPORT FAIRNESS INDEX/*"
report_fi = pd.DataFrame({})
report_fi["CTT"] = []

In [98]:
# Calcolo fairness index
nominal_throughputs = [11000, 8000, 5000, 5000, 8000, 11000, 5000, 8000, 11000]
nominal_throughputs = [x / 60 for x in nominal_throughputs]

measured_throughputs = []

for val in sorted(glob(root)):
    data = pd.read_csv(val)
    
    duration = (data["timeStamp"].max() - data["timeStamp"].min())/1000 
    throughput = data[data["responseMessage"] == "OK"].shape[0]/(duration)

    measured_throughputs.append(throughput)

normalized_throughputs = [m / n for m, n in zip(measured_throughputs, nominal_throughputs)]

sums = [sum(normalized_throughputs[i:i+3]) for i in range(0, len(normalized_throughputs), 3)]
square_of_sums = np.power(sums, 2)

squares = np.power(normalized_throughputs,2)
sum_of_squares = [sum(squares[i:i+3]) for i in range(0, len(squares), 3)]

fairness_indexes = [(m)/(3*n) for m, n in zip(square_of_sums, sum_of_squares)]

print("FAIRNESS INDEX FAIR:", fairness_indexes[0])
print("FAIRNESS INDEX UNFAIR:", fairness_indexes[2])
print("FAIRNESS INDEX RANDOM:", fairness_indexes[1])



FAIRNESS INDEX FAIR: 0.8333388769222038
FAIRNESS INDEX UNFAIR: 0.732656958759833
FAIRNESS INDEX RANDOM: 0.7813459487235578
